# Análisis Automatizado de Datos con IA
## Versión 1: Análisis Exploratorio + Generación de Insights con Claude

**Este notebook analiza automáticamente cualquier archivo CSV y genera un reporte ejecutivo usando la API de Claude.**

### Requisitos del dataset:
- Mínimo 2,000 registros
- Mínimo 10 columnas
- Puede contener variables numéricas y categóricas

---
## CONFIGURACIÓN INICIAL
### 👇 MODIFICA SOLO ESTA CELDA 👇

In [ ]:
# ============================================================================
# CONFIGURACIÓN - MODIFICA ESTOS VALORES
# ============================================================================

# Ruta al archivo CSV
CSV_PATH = "data/tu_archivo.csv"

# Descripción breve del dataset (para que Claude entienda el contexto)
DATASET_DESCRIPTION = """
Describe aquí tu dataset en 2-3 oraciones.
Por ejemplo: "Dataset de calidad de vinos blancos con propiedades fisicoquímicas.
La variable objetivo es 'quality' que indica la calidad del vino en escala del 1-10."
"""

# Nombre del dataset (para los reportes)
DATASET_NAME = "Mi Dataset"

# Variable objetivo (opcional - dejar None para detección automática)
# Si tu dataset tiene una variable objetivo específica, ponla aquí
TARGET_VARIABLE = None  # Ejemplo: "quality" o "precio" o None

# Separador del CSV (coma por defecto)
CSV_SEPARATOR = ","

# ============================================================================
print(" Configuración cargada")
print(f"    Archivo: {CSV_PATH}")
print(f"    Dataset: {DATASET_NAME}")

---
## 1- Importar Librerías

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import os
from dotenv import load_dotenv

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Cargar variables de entorno
load_dotenv()

# Crear directorios
os.makedirs('outputs', exist_ok=True)
os.makedirs('reports', exist_ok=True)

print("Librerías cargadas correctamente")

---
## 2- Cargar y Validar Datos

In [ ]:
# Cargar el dataset
print("Cargando dataset...")
df = pd.read_csv(CSV_PATH, sep=CSV_SEPARATOR)

# Limpiar columnas vacías
df = df.dropna(axis=1, how='all')
df.columns = df.columns.str.strip()

print(f"Dataset cargado: {df.shape[0]:,} registros x {df.shape[1]} columnas")

In [ ]:
# Validación de requisitos
print("="*60)
print("VALIDACIÓN DEL DATASET")
print("="*60)

requisitos_cumplidos = True

# Verificar registros
if df.shape[0] >= 2000:
    print(f"Registros: {df.shape[0]:,} (mínimo 2,000)")
else:
    print(f"Registros: {df.shape[0]:,} (mínimo 2,000)")
    requisitos_cumplidos = False

# Verificar columnas
if df.shape[1] >= 10:
    print(f"Columnas: {df.shape[1]} (mínimo 10)")
else:
    print(f"Columnas: {df.shape[1]} (mínimo 10)")
    requisitos_cumplidos = False

if requisitos_cumplidos:
    print("\n¡Dataset válido! Continuando con el análisis...")
else:
    print("\nEl dataset no cumple con los requisitos mínimos.")

In [ ]:
# Identificar tipos de columnas automáticamente
print("\n" + "="*60)
print("IDENTIFICACIÓN AUTOMÁTICA DE VARIABLES")
print("="*60)

# Columnas numéricas
columnas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()

# Columnas categóricas (object, category, o numéricas con pocos valores únicos)
columnas_categoricas = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Detectar numéricas que podrían ser categóricas (pocos valores únicos)
for col in columnas_numericas.copy():
    if df[col].nunique() <= 10:  # Si tiene 10 o menos valores únicos
        columnas_categoricas.append(col)
        
# Variables continuas (numéricas que no son categóricas)
columnas_continuas = [col for col in columnas_numericas if col not in columnas_categoricas]

print(f"\nVariables numéricas continuas ({len(columnas_continuas)}):")
for col in columnas_continuas:
    print(f"   • {col}")

print(f"\nVariables categóricas ({len(columnas_categoricas)}):")
for col in columnas_categoricas:
    print(f"   • {col} ({df[col].nunique()} valores únicos)")

# Detectar variable objetivo
if TARGET_VARIABLE and TARGET_VARIABLE in df.columns:
    variable_objetivo = TARGET_VARIABLE
elif columnas_categoricas:
    # Usar la última columna categórica como objetivo por defecto
    variable_objetivo = columnas_categoricas[-1]
else:
    variable_objetivo = None

print(f"\nVariable objetivo detectada: {variable_objetivo}")

In [ ]:
# Vista previa del dataset
print("\nVista previa del dataset:")
df.head(10)

In [ ]:
# Información del dataset
print("\nInformación del dataset:")
df.info()

---
## 3- Análisis de Valores Faltantes

In [ ]:
# Análisis de valores faltantes
print("="*60)
print("ANÁLISIS DE VALORES FALTANTES")
print("="*60)

valores_faltantes = df.isnull().sum()
porcentaje_faltantes = (df.isnull().sum() / len(df)) * 100

df_faltantes = pd.DataFrame({
    'Columna': valores_faltantes.index,
    'Faltantes': valores_faltantes.values,
    'Porcentaje (%)': porcentaje_faltantes.values
}).sort_values('Faltantes', ascending=False)

print(f"\nTotal de celdas: {df.shape[0] * df.shape[1]:,}")
print(f"Celdas con valores faltantes: {df.isnull().sum().sum():,}")
print(f"Porcentaje promedio de faltantes: {porcentaje_faltantes.mean():.2f}%")

# Mostrar solo columnas con faltantes
df_con_faltantes = df_faltantes[df_faltantes['Faltantes'] > 0]
if len(df_con_faltantes) > 0:
    print("\nColumnas con valores faltantes:")
    display(df_con_faltantes)
else:
    print("\nNo hay valores faltantes en el dataset")

In [ ]:
# Heatmap de valores faltantes
fig, ax = plt.subplots(figsize=(14, 6))
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis', ax=ax)
ax.set_title(f'Mapa de Valores Faltantes - {DATASET_NAME}', fontsize=14, fontweight='bold')
ax.set_xlabel('Variables')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('outputs/01_heatmap_valores_faltantes.png', dpi=150, bbox_inches='tight')
plt.show()
print("Guardado: outputs/01_heatmap_valores_faltantes.png")

---
## 4- Análisis Estadístico Descriptivo

In [ ]:
# Estadísticas descriptivas para variables numéricas
print("="*60)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS")
print("="*60)

if columnas_continuas:
    estadisticas = df[columnas_continuas].describe().T
    estadisticas['mediana'] = df[columnas_continuas].median()
    estadisticas['moda'] = df[columnas_continuas].mode().iloc[0]
    estadisticas['varianza'] = df[columnas_continuas].var()
    estadisticas['rango'] = estadisticas['max'] - estadisticas['min']
    estadisticas['coef_var (%)'] = (estadisticas['std'] / estadisticas['mean']) * 100
    
    display(estadisticas.round(4))
else:
    print("No hay variables numéricas continuas para analizar.")
    estadisticas = pd.DataFrame()

In [ ]:
# Estadísticas para variables categóricas
print("\n" + "="*60)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES CATEGÓRICAS")
print("="*60)

resumen_categoricas = {}

for col in columnas_categoricas:
    frecuencias = df[col].value_counts()
    porcentajes = (frecuencias / len(df) * 100).round(2)
    
    resumen_categoricas[col] = {
        'valores_unicos': df[col].nunique(),
        'moda': df[col].mode()[0] if len(df[col].mode()) > 0 else None,
        'frecuencia_moda': frecuencias.iloc[0] if len(frecuencias) > 0 else 0,
        'porcentaje_moda': porcentajes.iloc[0] if len(porcentajes) > 0 else 0
    }
    
    print(f"\n{col}:")
    print(f"   Valores únicos: {df[col].nunique()}")
    print(f"   Moda: {resumen_categoricas[col]['moda']} ({resumen_categoricas[col]['porcentaje_moda']}%)")
    print(f"   Distribución:")
    for val, freq, pct in zip(frecuencias.index[:5], frecuencias.values[:5], porcentajes.values[:5]):
        print(f"      • {val}: {freq:,} ({pct}%)")
    if len(frecuencias) > 5:
        print(f"      ... y {len(frecuencias) - 5} valores más")

---
## 5- Visualizaciones

In [ ]:
# Histogramas para variables numéricas
if columnas_continuas:
    n_cols = 3
    n_rows = (len(columnas_continuas) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 and n_cols == 1 else axes
    
    for i, col in enumerate(columnas_continuas):
        ax = axes[i]
        ax.hist(df[col].dropna(), bins=30, edgecolor='black', alpha=0.7, color='steelblue')
        ax.axvline(df[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df[col].mean():.2f}')
        ax.axvline(df[col].median(), color='green', linestyle=':', linewidth=2, label=f'Mediana: {df[col].median():.2f}')
        ax.set_title(col, fontsize=11, fontweight='bold')
        ax.legend(fontsize=8)
    
    # Ocultar ejes vacíos
    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)
    
    plt.suptitle(f'Histogramas - {DATASET_NAME}', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('outputs/02_histogramas.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Guardado: outputs/02_histogramas.png")
else:
    print("No hay variables numéricas para crear histogramas.")

In [ ]:
# Boxplots para variables numéricas
if columnas_continuas:
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 and n_cols == 1 else axes
    
    for i, col in enumerate(columnas_continuas):
        ax = axes[i]
        bp = ax.boxplot(df[col].dropna(), patch_artist=True)
        bp['boxes'][0].set_facecolor('lightblue')
        ax.set_title(col, fontsize=11, fontweight='bold')
    
    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)
    
    plt.suptitle(f'Boxplots (Detección de Outliers) - {DATASET_NAME}', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('outputs/03_boxplots.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Guardado: outputs/03_boxplots.png")

In [ ]:
# Gráficos de barras para variables categóricas
if columnas_categoricas:
    n_cat = min(len(columnas_categoricas), 6)  # Máximo 6 gráficos
    n_cols_cat = min(3, n_cat)
    n_rows_cat = (n_cat + n_cols_cat - 1) // n_cols_cat
    
    fig, axes = plt.subplots(n_rows_cat, n_cols_cat, figsize=(15, 4*n_rows_cat))
    if n_cat == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    for i, col in enumerate(columnas_categoricas[:n_cat]):
        ax = axes[i]
        frecuencias = df[col].value_counts().head(10)  # Top 10 valores
        colores = sns.color_palette('husl', n_colors=len(frecuencias))
        
        bars = ax.bar(frecuencias.index.astype(str), frecuencias.values, color=colores, edgecolor='black')
        ax.set_title(col, fontsize=11, fontweight='bold')
        ax.set_xlabel('')
        ax.set_ylabel('Frecuencia')
        plt.sca(ax)
        plt.xticks(rotation=45, ha='right')
    
    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)
    
    plt.suptitle(f'Distribución de Variables Categóricas - {DATASET_NAME}', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('outputs/04_barras_categoricas.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Guardado: outputs/04_barras_categoricas.png")
else:
    print("No hay variables categóricas para crear gráficos de barras.")

---
## 6- Detección de Outliers

In [ ]:
# Detección de outliers usando método IQR
print("="*60)
print("DETECCIÓN DE OUTLIERS (Método IQR)")
print("="*60)

def contar_outliers_iqr(columna):
    Q1 = columna.quantile(0.25)
    Q3 = columna.quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    outliers = ((columna < limite_inferior) | (columna > limite_superior)).sum()
    return outliers, limite_inferior, limite_superior

resultados_outliers = []
total_outliers = 0

for col in columnas_continuas:
    n_outliers, lim_inf, lim_sup = contar_outliers_iqr(df[col].dropna())
    pct_outliers = (n_outliers / len(df)) * 100
    total_outliers += n_outliers
    
    resultados_outliers.append({
        'Variable': col,
        'Outliers': n_outliers,
        'Porcentaje (%)': pct_outliers,
        'Límite Inf.': lim_inf,
        'Límite Sup.': lim_sup
    })

df_outliers = pd.DataFrame(resultados_outliers).sort_values('Outliers', ascending=False)
print(f"\nTotal de outliers detectados: {total_outliers:,}")
display(df_outliers)

---
## 7- Análisis de Correlaciones

In [ ]:
# Matriz de correlación
if len(columnas_numericas) > 1:
    print("="*60)
    print("ANÁLISIS DE CORRELACIONES")
    print("="*60)
    
    matriz_correlacion = df[columnas_numericas].corr(method='pearson')
    
    # Heatmap
    fig, ax = plt.subplots(figsize=(12, 10))
    mask = np.triu(np.ones_like(matriz_correlacion, dtype=bool))
    sns.heatmap(matriz_correlacion, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', 
                center=0, square=True, linewidths=0.5, ax=ax, cbar_kws={'shrink': 0.8})
    ax.set_title(f'Matriz de Correlaciones - {DATASET_NAME}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('outputs/05_heatmap_correlaciones.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Guardado: outputs/05_heatmap_correlaciones.png")
else:
    print("No hay suficientes variables numéricas para análisis de correlaciones.")
    matriz_correlacion = pd.DataFrame()

In [ ]:
# Top correlaciones
if len(columnas_numericas) > 1:
    correlaciones_pares = []
    for i in range(len(matriz_correlacion.columns)):
        for j in range(i+1, len(matriz_correlacion.columns)):
            col1 = matriz_correlacion.columns[i]
            col2 = matriz_correlacion.columns[j]
            corr = matriz_correlacion.iloc[i, j]
            correlaciones_pares.append((col1, col2, corr, abs(corr)))
    
    correlaciones_pares.sort(key=lambda x: x[3], reverse=True)
    top_correlaciones = correlaciones_pares[:10]
    
    print("\nTOP 10 CORRELACIONES MÁS FUERTES:")
    df_top_corr = pd.DataFrame(top_correlaciones, columns=['Variable 1', 'Variable 2', 'Correlación', '|Correlación|'])
    df_top_corr.index = range(1, len(df_top_corr) + 1)
    display(df_top_corr)
else:
    top_correlaciones = []
    df_top_corr = pd.DataFrame()

In [ ]:
# Correlaciones con variable objetivo (si existe)
if variable_objetivo and variable_objetivo in columnas_numericas:
    print(f"\nCORRELACIONES CON VARIABLE OBJETIVO ({variable_objetivo}):")
    corr_con_objetivo = matriz_correlacion[variable_objetivo].drop(variable_objetivo).sort_values(key=abs, ascending=False)
    
    df_corr_objetivo = pd.DataFrame({
        'Variable': corr_con_objetivo.index,
        'Correlación': corr_con_objetivo.values
    })
    df_corr_objetivo.index = range(1, len(df_corr_objetivo) + 1)
    display(df_corr_objetivo)
    
    # Gráfico de barras
    fig, ax = plt.subplots(figsize=(10, 6))
    colors = ['green' if c > 0 else 'red' for c in corr_con_objetivo.values]
    ax.barh(corr_con_objetivo.index, corr_con_objetivo.values, color=colors, edgecolor='black', alpha=0.7)
    ax.axvline(x=0, color='black', linewidth=0.5)
    ax.set_xlabel('Correlación')
    ax.set_title(f'Correlación con {variable_objetivo}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('outputs/06_correlacion_objetivo.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Guardado: outputs/06_correlacion_objetivo.png")
else:
    df_corr_objetivo = pd.DataFrame()
    print("\nNo se detectó variable objetivo numérica para análisis de correlación.")

---
## 8- Preparar Resumen para Claude

In [ ]:
# Construir resumen completo del análisis
print("="*60)
print("GENERANDO RESUMEN DEL ANÁLISIS")
print("="*60)

# Resumen de variable objetivo si existe
resumen_objetivo = ""
if variable_objetivo:
    if variable_objetivo in columnas_categoricas:
        freq_obj = df[variable_objetivo].value_counts()
        pct_obj = (freq_obj / len(df) * 100).round(2)
        resumen_objetivo = f"""
   Variable objetivo: {variable_objetivo}
   Tipo: Categórica
   Valores únicos: {df[variable_objetivo].nunique()}
   Distribución:
{chr(10).join([f'      - {val}: {freq:,} ({pct}%)' for val, freq, pct in zip(freq_obj.index[:10], freq_obj.values[:10], pct_obj.values[:10])])}
"""
    else:
        resumen_objetivo = f"""
   Variable objetivo: {variable_objetivo}
   Tipo: Numérica
   Media: {df[variable_objetivo].mean():.4f}
   Mediana: {df[variable_objetivo].median():.4f}
   Std: {df[variable_objetivo].std():.4f}
   Rango: [{df[variable_objetivo].min():.4f}, {df[variable_objetivo].max():.4f}]
"""

resumen_analisis = f"""
================================================================================
RESUMEN DEL ANÁLISIS DE DATOS
================================================================================

DESCRIPCIÓN DEL DATASET:
{DATASET_DESCRIPTION}

1. INFORMACIÓN GENERAL
--------------------------------------------------------------------------------
   - Nombre: {DATASET_NAME}
   - Dimensiones: {df.shape[0]:,} registros x {df.shape[1]} columnas
   - Variables numéricas continuas: {len(columnas_continuas)}
   - Variables categóricas: {len(columnas_categoricas)}
   - Duplicados: {df.duplicated().sum():,}

2. CALIDAD DE DATOS
--------------------------------------------------------------------------------
   - Valores faltantes totales: {df.isnull().sum().sum():,} ({porcentaje_faltantes.mean():.2f}% promedio)
   - Columnas con faltantes: {(valores_faltantes > 0).sum()}
   - Total de outliers (IQR): {total_outliers:,}

3. VARIABLE OBJETIVO
--------------------------------------------------------------------------------
{resumen_objetivo if resumen_objetivo else '   No se identificó variable objetivo específica.'}

4. ESTADÍSTICAS DESCRIPTIVAS (Variables Numéricas)
--------------------------------------------------------------------------------
{estadisticas[['mean', 'std', 'min', 'max', 'mediana']].to_string() if len(estadisticas) > 0 else 'No hay variables numéricas.'}

5. VARIABLES CATEGÓRICAS
--------------------------------------------------------------------------------
{chr(10).join([f"   - {col}: {info['valores_unicos']} valores únicos, moda: {info['moda']} ({info['porcentaje_moda']}%)" for col, info in resumen_categoricas.items()])}

6. OUTLIERS DETECTADOS (Top 5)
--------------------------------------------------------------------------------
{df_outliers.head(5).to_string() if len(df_outliers) > 0 else 'No hay outliers detectados.'}

7. TOP 5 CORRELACIONES MÁS FUERTES
--------------------------------------------------------------------------------
{df_top_corr.head(5).to_string() if len(df_top_corr) > 0 else 'No hay suficientes variables para correlaciones.'}

8. CORRELACIONES CON VARIABLE OBJETIVO
--------------------------------------------------------------------------------
{df_corr_objetivo.to_string() if len(df_corr_objetivo) > 0 else 'No aplica o no hay variable objetivo numérica.'}

9. OBSERVACIONES INICIALES
--------------------------------------------------------------------------------
   - El dataset tiene {df.shape[0]:,} registros y {df.shape[1]} variables
   - {'Hay ' + str(df.isnull().sum().sum()) + ' valores faltantes que requieren tratamiento' if df.isnull().sum().sum() > 0 else 'No hay valores faltantes'}
   - Se detectaron {total_outliers:,} outliers en las variables numéricas
   - {'La correlación más fuerte es entre ' + top_correlaciones[0][0] + ' y ' + top_correlaciones[0][1] + ' (' + str(round(top_correlaciones[0][2], 3)) + ')' if top_correlaciones else 'No se calcularon correlaciones'}
"""

print(resumen_analisis)

# Guardar resumen
with open('reports/resumen_analisis.txt', 'w', encoding='utf-8') as f:
    f.write(resumen_analisis)
print("\nResumen guardado en: reports/resumen_analisis.txt")

---
## 9- Generación de Insights con Claude API

In [ ]:
# Configurar cliente de Anthropic
from anthropic import Anthropic

api_key = os.getenv("ANTHROPIC_API_KEY")

if not api_key:
    print("ERROR: No se encontró ANTHROPIC_API_KEY")
    print("   Por favor, configura tu API key en el archivo .env")
else:
    print("API Key encontrada")
    print(f"   Key: {api_key[:15]}...")

In [ ]:
# Función para generar insights con Claude
def generar_insights_con_claude(resumen, descripcion, nombre_dataset, api_key):
    """
    Envía el resumen del análisis a Claude y obtiene insights profesionales.
    """
    client = Anthropic(api_key=api_key)
    
    prompt = f"""
Eres un experto en análisis de datos y ciencia de datos. 
A continuación te presento el resumen de un análisis exploratorio de datos.

CONTEXTO DEL DATASET:
{descripcion}

Por favor, genera un REPORTE EJECUTIVO PROFESIONAL que incluya:

1. RESUMEN EJECUTIVO (2-3 párrafos)
   - Descripción general del dataset
   - Calidad de los datos
   - Principales características observadas

2. HALLAZGOS CLAVE (3-5 hallazgos)
   - Insights importantes descubiertos en el análisis
   - Patrones o tendencias relevantes
   - Relaciones significativas entre variables

3. CALIDAD DE LOS DATOS
   - Evaluación de valores faltantes
   - Análisis de outliers detectados
   - Recomendaciones de limpieza

4. RECOMENDACIONES DE PREPROCESAMIENTO (3-5 recomendaciones)
   - Sugerencias para mejorar la calidad de los datos
   - Técnicas recomendadas para manejar outliers
   - Transformaciones sugeridas para modelado

5. CONCLUSIONES Y PRÓXIMOS PASOS
   - Resumen de conclusiones principales
   - Recomendaciones para análisis futuros
   - Posibles aplicaciones del análisis

DATOS DEL ANÁLISIS:
{resumen}

Por favor, responde en español y de manera profesional. Usa los datos específicos 
del resumen para fundamentar cada punto. Sé específico y evita generalidades.
"""
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=4000,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.content[0].text

In [ ]:
# Generar insights con Claude
if api_key:
    print("Generando insights con Claude...")
    print("   (Esto puede tomar unos segundos)\n")
    
    try:
        insights = generar_insights_con_claude(
            resumen_analisis, 
            DATASET_DESCRIPTION, 
            DATASET_NAME, 
            api_key
        )
        
        print("="*70)
        print("REPORTE EJECUTIVO GENERADO POR CLAUDE")
        print("="*70)
        print(insights)
        
        # Guardar insights
        with open('reports/insights_claude.txt', 'w', encoding='utf-8') as f:
            f.write(f"REPORTE EJECUTIVO - {DATASET_NAME}\n")
            f.write("="*70 + "\n\n")
            f.write(insights)
        
        print("\nInsights guardados en: reports/insights_claude.txt")
        
    except Exception as e:
        print(f"Error al generar insights: {e}")
else:
    print("No se puede generar insights sin API key.")

---
## Resumen Final

In [ ]:
# Resumen final del análisis
print("="*70)
print("ANÁLISIS COMPLETADO")
print("="*70)

print(f"""
DATASET: {DATASET_NAME}
ARCHIVO: {CSV_PATH}
DIMENSIONES: {df.shape[0]:,} registros x {df.shape[1]} columnas

   ARCHIVOS GENERADOS:
   Visualizaciones (outputs/):
   • 01_heatmap_valores_faltantes.png
   • 02_histogramas.png
   • 03_boxplots.png
   • 04_barras_categoricas.png
   • 05_heatmap_correlaciones.png
   • 06_correlacion_objetivo.png
   
   Reportes (reports/):
   • resumen_analisis.txt
   • insights_claude.txt

   MÉTRICAS CLAVE:
   • Variables numéricas: {len(columnas_continuas)}
   • Variables categóricas: {len(columnas_categoricas)}
   • Valores faltantes: {df.isnull().sum().sum():,}
   • Outliers detectados: {total_outliers:,}
   • Variable objetivo: {variable_objetivo}
""")

print("\n¡Análisis completado exitosamente!")